In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.8 MB/s eta 0:00:00


To set your API key, follow these steps:

1. Click on the "🔑" icon in the left sidebar.
2. Click on "Manage secrets in Colab".
3. Click "Add new secret".
4. For the "Name", enter `GROQ_API_KEY`.
5. For the "Value", enter your Groq API key.
6. Make sure the "Notebook access" toggle is turned on.

Once you have added the secret, you can access it in your code using `userdata.get('GROQ_API_KEY')`. I will update the code to use this method.

In [ ]:
from google.colab import userdata
import os
from groq import Groq

# Make sure you set your API key in the environment
# export GROQ_API_KEY="your_api_key_here"
api_key = userdata.get('GROQ_API_KEY')

print(f"API Key loaded: {api_key is not None}")

if not api_key:
    raise ValueError("Please set your GROQ_API_KEY environment variable in Colab secrets")

client = Groq(api_key=api_key)

# Chat completion example
chat_completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",  # You can choose from available models: llama3, gemma, mixtral, etc.
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the first cricket world cup?"},
    ],
    temperature=0.7,
    max_tokens=200,
)

print(chat_completion.choices[0].message.content)

API Key loaded: True
The first Cricket World Cup took place in 1975 in England. The West Indies cricket team, led by Clive Lloyd, won the tournament by defeating Australia in the final on June 21, 1975, at Lord's Cricket Ground in London.


In [ ]:
import langchain

In [ ]:
student_description="Mahendra Singh Dhoni born 7 July 1981 is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most prolific wicket-keeper batsmen and captains, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014."

In [ ]:
student_description

'Mahendra Singh Dhoni born 7 July 1981 is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most prolific wicket-keeper batsmen and captains, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014.'

# A simple prompt to extract information from "student_description" in a JSON format.

In [ ]:
from groq import Groq
client=Groq(api_key=api_key)

In [ ]:
response=client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user",
         "content": student_description
         },
    ],
)

In [ ]:
output=response.choices[0].message.content

In [ ]:
import json
# The model's output is not in JSON format, so we can't parse it as JSON.
# Instead, we can just display the raw output.
print(output)

That's partially correct. Mahendra Singh Dhoni, also known as MS Dhoni, was indeed born on 7 July 1981. However, he is also known for bowling legspin, not just batting. Dhoni can bowl, is an occasional leg-spinner, and has taken wickets in international cricket. 

MS Dhoni's accomplishments as a cricketer are extensive. He was a highly skilled wicket-keeper batsman and an effective skipper, having led the Indian national team to numerous successes, including:

- 2007 - Led India to their first T20 World Cup title.
- 2010 - Led India to their second T20 World Cup title.
- 2011 - Led India to the World Cup title, their first in 28 years, and also led the side in a historic series win in Australia.
- 2013 - Led India to a 4-0 series win in Australia, their first series victory in Australia in 32 years.

Dhoni also held the record for the most One Day International (ODI) stumpings (133) and most wicket-keeper dismissals in ODIs overall.


In [ ]:
student_custom_function=[
    {
        'type': 'function',
        'function': {
            'name': 'extaract_student_info',
            'description': 'Get the student information from the body of the input text',
            'parameters':{
                'type': 'object',
                'properties': {
                    'name':{
                        'type': 'string',
                        'description': 'Name of the person.'
                    },
                    'college':{
                        'type': 'string',
                        'description': 'Name of the person.'
                    },
                    'grades':{
                        'type': 'integer',
                        'description': 'CGPA of the student.'
                    },
                    'club':{
                        'type': 'string',
                        'description': 'College club for extracurricular activities.'
                    }
                },
                'required': ['name', 'college', 'club'] # Made 'grades' optional
            }
        }

}]

In [ ]:
import json

# Prompt the model to output JSON matching your function schema
prompt_with_instruction = f"""
Extract student information in JSON format with the following fields:
- name
- college
- grades (optional)
- club

Text: {student_description}

Return the information as a JSON object.
"""

response2 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_with_instruction},
    ],
    temperature=0.7,
    max_tokens=300
)

# Extract text
model_output = response2.choices[0].message.content

# Convert the JSON text from the model into Python dict
try:
    student_info = json.loads(model_output)
except json.JSONDecodeError:
    student_info = {"error": "Model output is not valid JSON", "raw_output": model_output}

print(student_info)

{'error': 'Model output is not valid JSON', 'raw_output': 'However, the text you provided does not contain any student information. It is a biography of an Indian cricketer, Mahendra Singh Dhoni. \n\nIf you\'d like, I can provide an example of student information in JSON format:\n\n```json\n{\n  "name": "John Doe",\n  "college": "University of Toronto",\n  "grades": ["A", "B", "A-"],\n  "club": "University Debate Team"\n}\n```\n\nOr if you\'d like to add another student:\n\n```json\n[\n  {\n    "name": "John Doe",\n    "college": "University of Toronto",\n    "grades": ["A", "B", "A-"],\n    "club": "University Debate Team"\n  },\n  {\n    "name": "Jane Smith",\n    "college": "University of British Columbia",\n    "grades": ["B+", "A", "A-"],\n    "club": "University Rowing Team"\n  }\n]\n```'}


In [ ]:
response2

ChatCompletion(id='chatcmpl-c7e62b02-3ec7-4c34-adf3-85be7a584aa5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='However, the text you provided does not contain any student information. It is a biography of an Indian cricketer, Mahendra Singh Dhoni. \n\nIf you\'d like, I can provide an example of student information in JSON format:\n\n```json\n{\n  "name": "John Doe",\n  "college": "University of Toronto",\n  "grades": ["A", "B", "A-"],\n  "club": "University Debate Team"\n}\n```\n\nOr if you\'d like to add another student:\n\n```json\n[\n  {\n    "name": "John Doe",\n    "college": "University of Toronto",\n    "grades": ["A", "B", "A-"],\n    "club": "University Debate Team"\n  },\n  {\n    "name": "Jane Smith",\n    "college": "University of British Columbia",\n    "grades": ["B+", "A", "A-"],\n    "club": "University Rowing Team"\n  }\n]\n```', role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_call

In [ ]:
response2.choices[0].message.content

'However, the text you provided does not contain any student information. It is a biography of an Indian cricketer, Mahendra Singh Dhoni. \n\nIf you\'d like, I can provide an example of student information in JSON format:\n\n```json\n{\n  "name": "John Doe",\n  "college": "University of Toronto",\n  "grades": ["A", "B", "A-"],\n  "club": "University Debate Team"\n}\n```\n\nOr if you\'d like to add another student:\n\n```json\n[\n  {\n    "name": "John Doe",\n    "college": "University of Toronto",\n    "grades": ["A", "B", "A-"],\n    "club": "University Debate Team"\n  },\n  {\n    "name": "Jane Smith",\n    "college": "University of British Columbia",\n    "grades": ["B+", "A", "A-"],\n    "club": "University Rowing Team"\n  }\n]\n```'

In [ ]:
# Assuming 'response2' is the object from the model's reply
message = response2.choices[0].message

# Check if 'function_call' exists before trying to access its attributes
if message.function_call is not None:
    # If it exists, you can safely access the arguments
    function_name = message.function_call.name
    function_args = message.function_call.arguments
    print(f"Model called function: {function_name} with arguments: {function_args}")
else:
    # If it's None, it means the model returned a regular text response
    print(f"Model returned a text message: {message.content}")

Model returned a text message: However, the text you provided does not contain any student information. It is a biography of an Indian cricketer, Mahendra Singh Dhoni. 

If you'd like, I can provide an example of student information in JSON format:

```json
{
  "name": "John Doe",
  "college": "University of Toronto",
  "grades": ["A", "B", "A-"],
  "club": "University Debate Team"
}
```

Or if you'd like to add another student:

```json
[
  {
    "name": "John Doe",
    "college": "University of Toronto",
    "grades": ["A", "B", "A-"],
    "club": "University Debate Team"
  },
  {
    "name": "Jane Smith",
    "college": "University of British Columbia",
    "grades": ["B+", "A", "A-"],
    "club": "University Rowing Team"
  }
]
```


In [ ]:
# Assuming 'response2' is the object from the model's reply
message = response2.choices[0].message

# Check if 'function_call' exists before trying to access its attributes
if message.function_call is not None:
    # If it exists, you can safely access the arguments
    function_name = message.function_call.name
    function_args = message.function_call.arguments
    print(f"Model called function: {function_name}")
    # Now, safely attempt to load the JSON arguments and print their type
    try:
        parsed_args = json.loads(function_args)
        print(f"Type of parsed arguments: {type(parsed_args)}")
    except json.JSONDecodeError:
        print(f"Function arguments are not valid JSON: {function_args}")
else:
    # If it's None, it means the model returned a regular text response
    print(f"Model returned a text message. No function call was made.")

Model returned a text message. No function call was made.


In [ ]:
student_description

'Mahendra Singh Dhoni born 7 July 1981 is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most prolific wicket-keeper batsmen and captains, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014.'

In [ ]:
student_description_two="Virat Kohli ,born 5 November 1988) is an Indian international cricketer and the former captain of the Indian national cricket team. He is a right-handed batsman and an occasional medium-pace bowler. He currently represents Royal Challengers Bengaluru in the IPL and Delhi in domestic cricket."

In [ ]:
student_info=[student_description, student_description_two]
for stu in student_info:
  response=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {"role": "user",
           "content": stu
           },
      ],
      # The functions parameter is not standard for Groq's API.
      # To guide the model to output JSON, it's better to use prompt engineering.
      # I've removed the functions parameter and will rely on the prompt
      # in the next step to get the desired JSON output.
      # functions=student_custom_function,
      # function_call="auto",
  )
  print(response.choices[0].message.content)

That's correct about Mahendra Singh Dhoni, also known as MS Dhoni. However, I'd like to add a few more points about his illustrious cricket career: 

He is widely regarded as one of the greatest and most successful captains in the history of Indian cricket. Dhoni led India to victory in the 2007 ICC World Twenty20, the 2010 Asia Cup, and the 2011 ICC Cricket World Cup.

He made his international debut in 2004 and earned numerous accolades throughout his career, including the ICC ODI Player of the Year in 2008 and the Padma Bhushan, India's third-highest civilian honor, in 2009.

Dhoni stepped down as the captain of the Indian limited-overs teams in 2017 and the test team in 2014. He played his final test match in 2014 and last ODI in 2019. He hung up his boots from international cricket in August 2020.
I think there may be a small inaccuracy in that information. Virat Kohli is an Indian international cricketer and the former captain of the Indian national cricket team. He is indeed a r

In [ ]:
student_description_three="Rohit Gurunath Sharma (born 30 April 1987) is an Indian international cricketer and the captain of the India national team in ODIs. He is also a former captain in Tests and T20Is. He is widely regarded as one of the greatest opening batters of all time, and one of the best batsmen of his generation. He is a right-handed batsman who plays for Mumbai Indians in Indian Premier League and for Mumbai in domestic cricket. After the team's victory at the 2024 Men's T20 World Cup, he announced his retirement from T20Is."

In [ ]:
student_info=[student_description, student_description_two, student_description_three]
for stu in student_info:
  response=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {"role": "user",
           "content": stu
           },
      ],
      # The functions parameter is not standard for Groq's API.
      # To guide the model to output JSON, it's better to use prompt engineering.
      # I've removed the functions parameter and will rely on the prompt
      # in the next step to get the desired JSON output.
      # functions=student_custom_function,
      # function_call="auto",
  )
  print(response.choices[0].message.content)

You are referring to Mahendra Singh Dhoni, also known as MS Dhoni, a highly accomplished Indian cricketer. Here's a brief overview of his cricketing career:

1. **Early Life:** Mahendra Singh Dhoni was born on July 7, 1981, in Ranchi, Jharkhand (formerly a part of Bihar), India.
2. **International Career:** He made his international debut in 2004 and quickly established himself as a valuable asset to the Indian cricket team.
3. **Wicket-Keeping and Batting:** As you mentioned, Dhoni is widely regarded as one of the most prolific wicket-keeping batsmen in international cricket. He was also known for his ability to finish off matches with his aggressive batting style.
4. **Captaincy Career:** Dhoni was appointed captain of the Indian cricket team in limited-overs formats (ODIs and T20Is) in 2007 and in Test cricket in 2008. He led the team to numerous victories, including the 2007 T20 World Cup, 2010 T20 World Cup, and the 2011 ODI World Cup.
5. **Trophy Wins:** Under his captaincy, Indi

In [ ]:
functions=[student_custom_function[0], student_custom_function[0]]
student_info=[student_description, student_description_two, student_description_three]
for stu in student_info:
  response=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {"role": "user",
           "content": stu
           },
      ],
      # The functions parameter is not standard for Groq's API.
      # To guide the model to output JSON, it's better to use prompt engineering.
      # I've removed the functions parameter and will rely on the prompt
      # in the next step to get the desired JSON output.
      # functions=student_custom_function,
      # function_call="auto",
  )
  print(response.choices[0].message.content)

That's partially correct about Mahendra Singh Dhoni. 

He indeed played for the Indian cricket team, but the information about his test and limited-overs captaincy periods seems a bit off. 

Mahendra Singh Dhoni was the captain of the Indian team in limited overs formats from 2007 to 2017 however he wasn't the test captain until 2008 though, but his last stint as the test captaincy was actually in 2014.
Virat Kohli is indeed an Indian international cricketer and former captain of the Indian national cricket team. He is known for his exceptional batting skills and is a right-handed batsman. 

Here are a few additional points about Virat Kohli:

- Virat Kohli has also been a successful captain of the Indian national cricket team. He led the side to numerous victories and was known for his strong leadership skills.
- Kohli has been the recipient of several awards and recognitions, including the ICC Cricketer of the Year award in 2012 and the Sir Garfield Sobers Trophy for the ICC Crickete

# Advance example of function calling

In [ ]:
response=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {
              "role": "user",
              "content": "When's the next flight from Delhi to Mumbai?"
           }
      ])

In [ ]:
response.choices[0].message.content

"I'd be happy to help you find flight information from Delhi to Mumbai. However, I'm a large language model, I don't have real-time access to flight scheduling data. \n\nTo find the next available flight from Delhi to Mumbai, I recommend checking with airlines directly or using a flight search engine like Google Flights, Skyscanner, or Expedia. You can also visit the websites of airlines such as Air India, SpiceJet, or IndiGo, which operate frequently from Delhi to Mumbai.\n\nIf you provide me with a specific date or travel details, I can try to suggest some airlines and flight times available on that date. \n\nHere are some approximate flight frequencies from Delhi to Mumbai:\n\n- Air India: Almost every 30 minutes to 1 hour\n- IndiGo: Frequent flights, approximately every 30 minutes to 1 hour\n- SpiceJet: Multiple flights daily, approximately every 1-2 hours\n\nKeep in mind that flight schedules may vary depending on various factors like travel dates, airline availability, and season

In [ ]:
function_descriptions=[
    {
    "name": "get_flight_info",
    "description": "Get the flight information between two locations",
    "parameters":{
        "type": "object",
        "properties": {
            "loc_origin":{
                "type": "string",
                "description": "The departure airport, e.g. DEL",
            },
            "loc_destination":{
                "type": "string",
                "description": "The destination airport, e.g. MUM",
            },

            },
    }
}]

In [ ]:
user_prompt="When's the next flight from New Delhi to Mumbai?"

In [ ]:
response2=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {
              "role": "user",
              "content": "When's the next flight from Delhi to Mumbai?"
           }
      ],
      # Add function calling
      functions=function_descriptions, # Pass the list of function descriptions
      function_call="auto", #specify the function call
      )

In [ ]:
response2.choices[0].message

ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=FunctionCall(arguments='{"loc_destination":"BOM","loc_origin":"DEL"}', name='get_flight_info'), reasoning=None, tool_calls=None)

In [ ]:
response2.choices[0].message.function_call.arguments

'{"loc_destination":"BOM","loc_origin":"DEL"}'

# Use API of Make my Trip to get answers about flights

In [ ]:
from datetime import datetime, timedelta
def get_flight_info(loc_origin, loc_destination):
  """Get flight information between two locations."""

  # Example output returned from an API or database
  flight_info={
      "loc_origin": loc_origin,
      "loc_destination": loc_destination,
      "datetime": str(datetime.now()+timedelta(hours=2)),
      "airline": "KLM",
      "flight": "KL643",
  }
  return json.dumps(flight_info)

In [ ]:
params=json.loads(response2.choices[0].message.function_call.arguments)

In [ ]:
json.loads(response2.choices[0].message.function_call.arguments).get("loc_origin")

'DEL'

In [ ]:
json.loads(response2.choices[0].message.function_call.arguments).get("loc_destination")

'BOM'

In [ ]:
origin=json.loads(response2.choices[0].message.function_call.arguments).get("loc_origin")
destination=json.loads(response2.choices[0].message.function_call.arguments).get("loc_destination")

In [ ]:
response2.choices[0].message.function_call.name

'get_flight_info'

In [ ]:
type(response2.choices[0].message.function_call.name)

str

In [ ]:
eval(response2.choices[0].message.function_call.name)

<function __main__.get_flight_info(loc_origin, loc_destination)>

In [ ]:
chosen_function = eval(response2.choices[0].message.function_call.name)
flight=chosen_function(**params)
print(flight)

{"loc_origin": "DEL", "loc_destination": "BOM", "datetime": "2025-10-03 07:13:58.354356", "airline": "KLM", "flight": "KL643"}


In [ ]:
user_prompt

"When's the next flight from New Delhi to Mumbai?"

In [ ]:
response2.choices[0].message.function_call.name

'get_flight_info'

In [ ]:
response3=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {"role": "user","content": user_prompt},
          {"role": "function", "name": response2.choices[0].message.function_call.name, "content": flight}
      ],
      # Add function calling
      functions=function_descriptions,
      function_call="auto", #specify the function call
      )

In [ ]:
response3

ChatCompletion(id='chatcmpl-0e26835b-18e0-494e-86a4-1c0b44d0ac1a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<function=get_flight_info>{"loc_origin": "DEL", "loc_destination": "BOM"}', role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1759468440, model='llama-3.1-8b-instant', object='chat.completion', system_fingerprint='fp_e32974efee', usage=CompletionUsage(completion_tokens=21, prompt_tokens=301, total_tokens=322, completion_time=0.031087194, prompt_time=0.0166431, queue_time=0.2963175, total_time=0.047730294), usage_breakdown=None, x_groq={'id': 'req_01k6m7vjjvemk9fdpwj3vtsbgw'}, service_tier='on_demand')

In [ ]:
response3.choices[0].message.content

'<function=get_flight_info>{"loc_origin": "DEL", "loc_destination": "BOM"}'

# Function Calling- Learn how to connect LLM's to External Tools.

In [ ]:
#Langchain
!pip install langchain_community

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
client= Groq(api_key=GROQ_API_KEY)

In [ ]:
# Zero shot prompting
prompt="Can you tell me the total number of countries in Asia? Can you give me a plain text list of the top 10 country names by population?"

In [ ]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt}
    ]
)
print(response.choices[0].message.content)

There are 49 countries in Asia, 2 of which are partially in Europe (Turkey and Russia) or partially in Africa (Egypt, but not in traditional geographical definitions), but sometimes, and depending on how you group and consider these countries and how you consider parts of countries outside certain continents in the traditional geographical definition of those continents; so, these numbers can change based on how you are counting and categorizing.

Here's a plain text list of the top 10 most populous countries in Asia:

1. China - approximately 1.45 billion people
2. India - approximately 1.41 billion people
3. Indonesia - approximately 284 million people
4. Pakistan - approximately 229 million people
5. Bangladesh - approximately 166 million people
6. Japan - approximately 128 million people
7. Philippines - approximately 112 million people
8. Vietnam - approximately 102 million people
9. Thailand - approximately 71 million people
10. North Korea - approximately 26 million people


In [ ]:
# Zero shot prompting
prompt2="Can you tell me capital of India?"

response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt2}
    ]
)
print(response.choices[0].message.content)

The capital of India is New Delhi.


In [ ]:
# Zero shot prompting
prompt3="What is vector and tokens ?"

response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt3}
    ]
)
print(response.choices[0].message.content)

**Vector and Tokens: A Brief Introduction**

In the context of computer science and artificial intelligence, vectors and tokens are fundamental concepts used in various applications, including natural language processing (NLP), machine learning (ML), and data analysis.

### Vectors

In mathematics and computer science, a **vector** is a mathematical object that represents a quantity with both magnitude and direction. In the context of NLP and ML, vectors are used to represent words, documents, or other text-based data in a high-dimensional space.

Think of a vector as a coordinate in a high-dimensional space, where each dimension represents a unique aspect of the data. For example, if we represent words as vectors, each dimension might correspond to a unique word or concept in the language.

Vectors have several key properties:

*   **Magnitude**: The length of the vector.
*   **Direction**: The direction of the vector in the high-dimensional space.
*   **Similarity**: Vectors with sim

# Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template_name=PromptTemplate(
    input_variables=["country"],
    template="Can you tell me capital of {country}?"
)

In [ ]:
prompt1=prompt_template_name.format(country="India")
response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt1}
    ]
)
print(response.choices[0].message.content)

The capital of India is New Delhi.


In [ ]:
prompt2=prompt_template_name.format(country="China")
response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt2}
    ]
)
print(response.choices[0].message.content)

The capital of China is Beijing.


In [ ]:
prompt=PromptTemplate.from_template("What is a good name for a company that makes {product}?")

In [ ]:
prompt.format(product="Chocolate")

'What is a good name for a company that makes Chocolate?'

In [ ]:
prompt3=prompt.format(product="Chocolate")
response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt3}
    ]
)
print(response.choices[0].message.content)

Here are some name suggestions for a chocolate company:

1. **ChocoCraft Co.**: This name suggests high-quality, artisanal chocolates.
2. **Cocoa Luxe**: This name evokes feelings of indulgence and luxury.
3. **SweetSpot Chocolates**: This name is catchy and implies that the chocolates will satisfy your sweet tooth.
4. **The Chocolate Studio**: This name suggests a creative, artistic approach to chocolate-making.
5. **ChocoBliss**: This name conveys feelings of happiness and satisfaction.
6. **Truffle Trove**: This name suggests a treasure trove of decadent, rich chocolates.
7. **GoldenNourish**: This name implies that the chocolates are not only delicious but also nourishing.
8. **Dark & Delicious**: This name highlights the rich, bold flavors of dark chocolate.
9. **ChocoHaven**: This name suggests a warm, inviting space where customers can indulge in chocolate treats.
10. **Meltdown Chocolates**: This name is playful and implies that the chocolates are so good, they'll melt in your 

# Agents

In [ ]:
prompt4="Can you tell me who won the recent T-20 Cricket World Cup?"
response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt4}
    ]
)
print(response.choices[0].message.content)

However, I need to clarify that I'm a large language model, my knowledge cutoff is December 2023, and information after that is not available to me.

If you are looking for the latest information on the T20 Cricket World Cup, I can tell you that the 2022 ICC Men's T20 World Cup was won by England. However, please note that the information may be outdated, and I do not have information on recent events.

For the most recent information, I recommend checking news websites, international cricket boards, or official cricket websites for the latest updates.


In [ ]:
prompt5="Can you tell me GDP of India?"
response = client.chat.completions.create(
    model="llama-3.1-8b-instant", # Or another suitable model
    messages=[
        {"role": "user", "content": prompt5}
    ]
)
print(response.choices[0].message.content)

The GDP (Gross Domestic Product) of India can be found by checking the most reliable sources such as the Reserve Bank of India (RBI) or the World Bank.

According to the latest available data from these sources as of my cut-off knowledge in 2023, the nominal GDP of India was approximately $3.53 trillion (or 2,44,67,000 crore Indian Rupees) for the year 2023.


# For extracting a real time info I am going to use Serp API
# Now by using this Serp API, I will call google-search-engine
# And I will extract the information in real time.

In [ ]:
#!pip install google-search-results langchain

In [ ]:
!pip install serpapi google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=e2b1472109e934673f855bddf557098e2263b4a083e525a52c74a261c4a65bb4
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [ ]:
!pip install google-search-results

In [ ]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain_groq import ChatGroq
import os
from google.colab import userdata

# Replace with your actual keys
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

llm = ChatGroq(
    groq_api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.1-8b-instant"
)
tools = load_tools(
    ["serpapi"],  # make sure SERPAPI_API_KEY is set
    llm=llm
)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
response = agent.run("Who is the Prime Minister of India?")
print(response)

ModuleNotFoundError: No module named 'langchain_groq'

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=17eba356f23d00e4982c24a25c30669cf38841af12cb6a74e47fd44519152b36
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [ ]:
!pip install -U langchain-groq

In [ ]:
tool=load_tools(["wikipedia"],llm=client)
llm=client

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model="llama-3.1-8b-instant"
)

agent=initialize_agent(
    tool,
    llm, # Use the ChatGroq instance here
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/tmp/ipython-input-2976241588.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(


In [ ]:
agent.run("Can you tell me more about this recent Cricket AsiaCup 2025?")

/tmp/ipython-input-1408538818.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Can you tell me more about this recent Cricket AsiaCup 2025?")




> Entering new AgentExecutor chain...
Thought: I need to find information about the recent Cricket AsiaCup 2025.
Action: wikipedia
Action Input: Cricket Asia Cup 2025
Observation: Page: 2025 Asia Cup
Summary: The 2025 Men's Asia Cup (also known as DP World Asia Cup for sponsorship reasons) was the 17th edition of the Asia Cup cricket tournament. It took place in the United Arab Emirates (UAE) from 9 to 28 September 2025. The matches were played in the Twenty20 International (T20I) format. India was the defending champions of 2023.  
The tournament featured eight teams. The five full members of the Asian Cricket Council, Afghanistan, Bangladesh, India, Pakistan, and Sri Lanka automatically qualified for the tournament and were joined by United Arab Emirates, Oman and Hong Kong, the teams which finished in the top three in the 2024 ACC Men's Premier Cup. In the final, India defeated Pakistan by 5 wickets to win their ninth title, and thus retained the title.

Page: Asia Cup
Summary: Th

'The 2025 Asia Cup was held in the United Arab Emirates from 9 to 28 September 2025, and India defeated Pakistan by 5 wickets to win their ninth title.'

In [ ]:
agent.run("Can you tell me what is GDP of India?")



> Entering new AgentExecutor chain...
Thought: To find the GDP of India, I need to look up information about India's economy.
Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in I

"The GDP of India is not explicitly mentioned in the provided text, but according to the information, India's economy is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP), and the country's domestic consumption drives nearly 70% of its GDP."

# Chain

In [ ]:
client

In [ ]:
from langchain.prompts import PromptTemplate
prompt=PromptTemplate.from_template("What is a good name for a company that makes {product}?")

In [ ]:
from langchain.chains import LLMChain

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=GROQ_API_KEY, model="llama-3.1-8b-instant")
chain=LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-688349982.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(llm=llm, prompt=prompt)


In [ ]:
chain.run("Wine").strip()

'Here are some suggestions for a wine company name:\n\n1. **Vinum Vita** (Latin for "Wine of Life"): This name conveys the idea that wine is an essential part of life and living.\n2. **Terroir Wine Co.**: "Terroir" refers to the unique characteristics of a wine\'s region or soil, making this name a great choice for a winery that emphasizes its connection to the land.\n3. **Vino Noir** (French for "Black Wine"): This name has a sleek, sophisticated sound and could work well for a winery that produces rich, full-bodied red wines.\n4. **Côte & Vine** (French for "Slope and Vine"): This name evokes the idea of rolling hills and vineyards, making it a great choice for a winery that produces high-quality, estate-grown wines.\n5. **Soleil Wine Co.** (French for "Sun Wine"): This name captures the warmth and sunshine of the wine-growing process and could work well for a winery that produces light, fruity wines.\n6. **Ridgeview Vineyards**: This name suggests a panoramic view of the vineyards, 

#Example 2

In [ ]:
prompt_template=PromptTemplate(
    input_variables=["Cuisine"],
    template="I want to open a restaurant for {cuisine} food, suggest a fancy name for this?"
)

In [ ]:
prompt_template

PromptTemplate(input_variables=['cuisine'], input_types={}, partial_variables={}, template='I want to open a restaurant for {cuisine} food, suggest a fancy name for this?')

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=GROQ_API_KEY, model="llama-3.1-8b-instant")
chain=LLMChain(llm=llm, prompt=prompt_template, verbose=True)

In [ ]:
chain.run("Indian").strip()



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Indian food, suggest a fancy name for this?

> Finished chain.


'Here are some fancy name suggestions for your Indian restaurant:\n\n1. **Mumbai Maison**: This name combines the vibrant city of Mumbai with the French word "maison," implying a sophisticated dining experience.\n2. **Tandoori Palace**: This name highlights the iconic tandoor oven, a staple of Indian cuisine, and suggests a regal, palace-like atmosphere.\n3. **Spice Route Elegance**: This name evokes the rich history of the spice trade and implies a luxurious, refined dining experience.\n4. **Desert Rose**: This name suggests a warm, exotic ambiance and could appeal to customers looking for a unique Indian dining experience.\n5. **Saffron & Silk**: This name incorporates two luxurious ingredients commonly used in Indian cuisine and implies a high-end, elegant atmosphere.\n6. **Dhaba Delights**: This name references the traditional Indian roadside eateries known as dhabas and suggests a warm, inviting atmosphere.\n7. **Maharaja\'s Kitchen**: This name references the rich history of Indi

In [ ]:
chain.run("American")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for American food, suggest a fancy name for this?

> Finished chain.


'Here are some fancy name suggestions for an American restaurant:\n\n1. **Liberty Bites**: This name evokes the spirit of American freedom and offers a playful twist on the idea of bite-sized food.\n2. **Main Street Eats**: This name has a nostalgic feel, reminiscent of classic American diners and the idea of casual, comforting food.\n3. **Star Spangled Table**: This name incorporates the American flag\'s iconic design and implies a sense of patriotism and tradition.\n4. **Red White & Grill**: This name combines the colors of the American flag with a playful nod to the idea of grilled food.\n5. **The American Dream Kitchen**: This name taps into the idea of the "American Dream" and suggests a warm, welcoming atmosphere.\n6. **Colonial Comforts**: This name references the early days of American history and implies a cozy, comforting dining experience.\n7. **Freedom Fork**: This name incorporates a playful take on the idea of freedom and offers a fun, casual vibe.\n8. **American Harvest*

# If you want to combine multiple chain and set a sequence for that we use simple sequential chain

In [ ]:

prompt_template_name=PromptTemplate(
    input_variables=["startup_name"],
    template="I want to start a startup for {startup-name}, suggest a good name for this?"
)
name_chain=LLMChain(llm=llm, prompt=prompt_template_name)
prompt_template_items=PromptTemplate(
    input_variables=["startup_name"],
    template="Suggest some items for {name}?"
)
strategies_chain=LLMChain(llm=llm, prompt=prompt_template_items)

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain, SequentialChain

strategies_chain=LLMChain(llm=llm, prompt=prompt_template_items, output_key="items")

chain = SequentialChain(
    chains=[name_chain, strategies_chain],
    input_variables=["startup_name"],
    output_variables=["startup_name", "items"]
)

ValidationError: 1 validation error for SequentialChain
  Value error, Missing required input keys: {'startup-name'}, only had {'startup_name'} [type=value_error, input_value={'chains': [LLMChain(verb...startup_name', 'items']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
print(chain.run("Artificial Intelligence. Respond in plain text only, no Markdown formatting."))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Artificial Intelligence. Respond in plain text only, no Markdown formatting. food, suggest a fancy name for this?

> Finished chain.
Some suggestions for a fancy name for an AI-themed restaurant could be:

1. Neurobyte
2. Cerebro Bistro
3. The Digital Diner
4. Byte & Blade
5. Synapse Kitchen
6. The Robo Bistro
7. Code & Crumb
8. Circuit Bites
9. The AI Cafe
10. Omega Eatery


In [ ]:
from langchain.prompts import PromptTemplate
prompt_template_name=PromptTemplate(
    input_variables=["startup_name"],
    template="I want to start a startup for {startup_name}, suggest a good name for this?"
)
name_chain=LLMChain(llm=llm, prompt=prompt_template_name, output_key="startup_name")
prompt_template_items=PromptTemplate(
    input_variables=["startup_name"],
    template="Suggest some items for {startup_name}?"
)

# Now lets try to understand the sequential chain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
#chain({"startup_name": "Artificial Intelligence"})

# Document loaders

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader=PyPDFLoader(r"/GFG_Nation_Skillup.pdf")

ValueError: File path /GFG_Nation_Skillup.pdf is not a valid file or url

In [ ]:
loader

NameError: name 'loader' is not defined

In [ ]:
pages=loader.load_and_split()
print(pages)

NameError: name 'loader' is not defined

# Memory

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
client=Groq(api_key=GROQ_API_KEY)

In [ ]:
prompt_template_name=PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?"
)

In [ ]:
chain=LLMChain(llm=llm, prompt=prompt_template_name)

In [ ]:
print(chain.run("Colour cup").strip())

Here are some suggestions for a company name that makes colorful cups:

1. **Hue & Co.**: A play on the word "hue," which refers to a color, and "Co.," short for company.
2. **ColorCup Co.**: Simple and straightforward, this name tells customers exactly what the company does.
3. **Vibrant Cups**: This name evokes the idea of bright, vibrant colors and could appeal to customers looking for fun and playful products.
4. **Chroma Cups**: "Chroma" is a term for the science of color, making this name a clever play on words.
5. **SplashCups**: This name suggests fun, vibrant colors and could appeal to customers looking for colorful and playful products.
6. **ColorCraft Co.**: This name suggests a focus on craftsmanship and attention to detail, which could appeal to customers looking for high-quality products.
7. **BrewBright**: This name suggests a connection to coffee or tea, while also emphasizing the bright, colorful aspect of the cups.
8. **ColorWise**: This name suggests a focus on under

In [ ]:
print(chain.run("Drones").strip())

Here are some suggestions for a drone company name:

1. **SkyGuard**: Suggests protection and surveillance.
2. **FlyTech**: Implies cutting-edge technology and flight capabilities.
3. **Aerius**: Latin for "relating to the air," conveying a sense of aerial expertise.
4. **Dronecraft Co.**: Emphasizes craftsmanship and quality control.
5. **Altitude Solutions**: Implies a focus on height and problem-solving.
6. **Skybound UAV**: Suggests limitless possibilities and aerial exploration.
7. **VoxAero**: Combines "vox" (voice) and "aero" (air), hinting at voice-controlled drones.
8. **Apex Aerial**: Conveys a sense of peak performance and aerial superiority.
9. **Flight Dynamics**: Highlights the company's expertise in drone movement and control.
10. **SkyPilot**: Suggests navigation and guidance, with a sense of adventure.
11. **Airborne Innovations**: Implies a focus on innovative technology and aerial exploration.
12. **DroneWorks**: Simple and straightforward, emphasizing the company's 

In [ ]:
type(chain.memory)

NoneType

# ConversationBufferMemory
# We can attach memory to remember all previous conversations

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory=ConversationBufferMemory()

/tmp/ipython-input-2290902444.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()


In [ ]:
chain=LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)

In [ ]:
print(chain.run("Wines").strip())

Naming a wine company can be a delightful task, as it involves combining creativity with the rich history and culture of wine. Here are some suggestions for a wine company name:

1. **Vino Vita** - "Vino" means wine in Italian, and "Vita" means life, conveying the idea of wine as a part of a lively and joyful lifestyle.
2. **Savor & Vine** - This name highlights the experience of savoring a fine wine and the importance of the vine in wine production.
3. **Terroir & Co.** - "Terroir" refers to the unique combination of environmental and geographical factors that affect a wine's taste and character.
4. **Wine Oak Estate** - This name evokes the idea of a prestigious, rustic wine estate with a focus on oak aging.
5. **La Vie En Rose** - This name, inspired by the classic song, conveys a sense of elegance and romance, perfect for a high-end wine company.
6. **Vineyard Vintners** - This name emphasizes the company's focus on vineyards and the art of winemaking.
7. **Grape Expectations** - A

In [ ]:
print(chain.run("Camera").strip())

Here are some suggestions for a camera company name:

1. **Lumina**: Inspired by the Latin word for light, conveying the idea of capturing the world's beauty through photographs.
2. **Clickspace**: A play on words combining "click" (as in, the sound of a camera shutter) with "space," suggesting a wide range of creative possibilities.
3. **FocusOn**: A straightforward name emphasizing the importance of focus in photography, with a friendly, approachable tone.
4. **Pixellum**: Combining "pixel" with "lum" (light), hinting at the high-quality images produced by the company's cameras.
5. **Apexion**: A combination of "apex" and "ion," suggesting a peak level of performance and innovation in camera technology.
6. **Snapview**: Emphasizing the instant, high-quality views captured by the company's cameras.
7. **FocalPoint**: Highlighting the idea of finding the perfect balance between aperture, shutter speed, and ISO to achieve stunning images.
8. **CaptureCraft**: Suggesting a company that n

In [ ]:
print(chain.run("Drones").strip())

Here are some ideas for a company name that makes drones:

1. **Aerius**: Derived from the Latin word for "relating to the air," this name conveys a sense of innovation and flight.
2. **SkyGuard**: Suggests a company that protects and monitors from above, implying a sense of security and vigilance.
3. **DroneCraft Co.**: Emphasizes the craftsmanship and expertise that goes into building high-quality drones.
4. **FlyTech Innovations**: Highlights the company's focus on innovation and technological advancements in drone design.
5. **Altitude Aerospace**: Conveys a sense of reach and exploration, suggesting a company that pushes the boundaries of what's possible with drones.
6. **SkyEye Robotics**: Implies a company that offers a bird's-eye view and a robotic approach to drone development.
7. **Dronius**: A playful name that sounds futuristic and innovative.
8. **FlyZone**: Suggests a company that offers a range of drone solutions for various applications and industries.
9. **Apex Aero**:

In [ ]:
chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Wines', additional_kwargs={}, response_metadata={}), AIMessage(content='Naming a wine company can be a delightful task, as it involves combining creativity with the rich history and culture of wine. Here are some suggestions for a wine company name:\n\n1. **Vino Vita** - "Vino" means wine in Italian, and "Vita" means life, conveying the idea of wine as a part of a lively and joyful lifestyle.\n2. **Savor & Vine** - This name highlights the experience of savoring a fine wine and the importance of the vine in wine production.\n3. **Terroir & Co.** - "Terroir" refers to the unique combination of environmental and geographical factors that affect a wine\'s taste and character.\n4. **Wine Oak Estate** - This name evokes the idea of a prestigious, rustic wine estate with a focus on oak aging.\n5. **La Vie En Rose** - This name, inspired by the classic song, conveys a sense of elegance and romance,

In [ ]:
print(chain.memory.buffer)

Human: Wines
AI: Naming a wine company can be a delightful task, as it involves combining creativity with the rich history and culture of wine. Here are some suggestions for a wine company name:

1. **Vino Vita** - "Vino" means wine in Italian, and "Vita" means life, conveying the idea of wine as a part of a lively and joyful lifestyle.
2. **Savor & Vine** - This name highlights the experience of savoring a fine wine and the importance of the vine in wine production.
3. **Terroir & Co.** - "Terroir" refers to the unique combination of environmental and geographical factors that affect a wine's taste and character.
4. **Wine Oak Estate** - This name evokes the idea of a prestigious, rustic wine estate with a focus on oak aging.
5. **La Vie En Rose** - This name, inspired by the classic song, conveys a sense of elegance and romance, perfect for a high-end wine company.
6. **Vineyard Vintners** - This name emphasizes the company's focus on vineyards and the art of winemaking.
7. **Grape E

#ConversationChain
## Conversation buffer memory goes growing endlessly.
## Just remember last 5 Conversation Chain
## Just remember last 10-20 Conversation Chain

In [ ]:
from langchain.chains import ConversationChain

In [ ]:
from langchain.chains import ConversationChain
from langchain_groq import ChatGroq

llm_with_temperature = ChatGroq(api_key=GROQ_API_KEY, temperature=0.7, model="llama-3.1-8b-instant")

convo=ConversationChain(llm=llm_with_temperature)

/tmp/ipython-input-215581644.py:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo=ConversationChain(llm=llm_with_temperature)


In [ ]:
convo.prompt

PromptTemplate(input_variables=['history', 'input'], input_types={}, partial_variables={}, template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [ ]:
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
convo.run("Who won the first cricket worldcup?")

"The first Cricket World Cup was held in England in 1975. It was a relatively short tournament, with only 8 teams participating. The matches were played on two different types of pitches - the Oval in London and the Headingley in Leeds. The final match was played at Lord's Cricket Ground in London, which is also known as the 'Home of Cricket'.\n\nAs for the winner, it was the West Indies team that emerged victorious. They defeated Australia in the final, with a score of 291/8 in 60 overs. The West Indies team was led by Clive Lloyd, who is considered one of the greatest cricketers of all time. His team's victory in the World Cup marked the beginning of a golden era for West Indian cricket.\n\nWould you like to know more about the history of the Cricket World Cup or its future developments?"

In [ ]:
convo.run("Can you tell me how much will 5+5?")

'A simple question after a detailed discussion about the Cricket World Cup. The answer to your math question is straightforward. 5 + 5 equals 10.'

In [ ]:
convo.run("Who was the captain of the winning team?")

"Now that we've discussed the Cricket World Cup, I'm glad to provide the answer to your next question. The captain of the winning West Indies team was indeed Clive Lloyd. He led his team to victory in the inaugural Cricket World Cup, which was a historic moment in cricket history. As a great leader and cricketer, Clive Lloyd is still revered by cricket enthusiasts around the world.\n\nBy the way, would you like to know more about Clive Lloyd's career or the impact he had on West Indian cricket?"

In [ ]:
convo.run("Can you divide the numbers and can you give me a final answer?")

"A quick math question to wrap up our conversation. To divide the numbers, we'll divide 5 by 5. 5 ÷ 5 equals 1.\n\nAs for a final answer, it seems we've covered quite a bit of ground, from the history of the Cricket World Cup to a simple math problem. If you'd like to end the conversation here, I'd be happy to bid you farewell. However, if you have any more questions or topics you'd like to discuss, I'm always here to help."

#ConversationBufferWindowMemory



In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory=ConversationBufferWindowMemory(k=1)
# Here, value of k is window size i.e. till how many prompts the bot can remember the answer

/tmp/ipython-input-3074714715.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferWindowMemory(k=1)


In [ ]:
convo=ConversationChain(llm=ChatGroq(api_key=GROQ_API_KEY, temperature=0.7, model="llama-3.1-8b-instant"),memory=memory)

In [ ]:
convo.run("Who won the first cricket worldcup?")

"What a fantastic question to start with. I'm delighted to share that the first Cricket World Cup was held in 1975 in England, and it was won by the West Indies cricket team. They defeated Australia in the final by 17 runs at Lord's Cricket Ground in London on June 21, 1975. The West Indies team was led by Clive Lloyd, and they were considered one of the greatest teams of all time, with legendary players like Vivian Richards, Andy Roberts, and Malcolm Marshall. What a thrilling match that must have been!"

In [ ]:
convo.run("Can you tell me how much will 5+5?")

"A simple yet crucial question to get back to basics. I'm happy to provide a straightforward answer. 5 + 5 equals 10."